In [1]:
import pandas as pd
import pyodbc
import datetime

In [2]:
# Connect to DataBank
connection_string = (
    "DSN=DataBank;"
    "Trusted_Connection=yes;"
)

con = pyodbc.connect(connection_string)

In [3]:
# Write sql query
qry = "SELECT application_reference, application_source, decision_owner, decision_date, opportunity_id, opportunity_name, opportunity_source, derived_status, application_status, organisation_source, amount_applied_for, joint_proposal, parent_award FROM ukri_ssd_snapshot.2025_04_24_ssd_application;"

# Read sql query
df= pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\3014746289.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql(qry, con)


In [4]:
# Write sql query
qry = "SELECT application_reference, application_source, organisation_id, organisation_source, participant_id, organisation_role FROM ukri_ssd_snapshot.2025_04_24_ssd_organisation_to_application;"

# Read sql query
organisation_to_application = pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\2147608080.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  organisation_to_application = pd.read_sql(qry, con)


In [5]:
data_merge = pd.merge(df, organisation_to_application, on=['application_reference', 'application_source'])

In [6]:
# Select decision date is not null or in the future

data_select = data_merge[~data_merge['decision_date'].isna() & (data_merge['decision_date'].dt.date < datetime.date.today()) & (data_merge['application_status'] != 'RO Withdrawn')]

In [7]:
datetime.date.today()

data_merge['decision_date'].dt.date > datetime.date.today()

data_merge['application_status'].unique()

array(['Rejected', 'Unsuccessful', 'Active', 'Accepted', 'Announced',
       None, 'Unsuccessful checks', 'Authorised', 'Passed checks',
       'Suspended Pending Activation', 'Submitted', 'RO Withdrawn',
       'Awaiting authorisation', 'Approved', 'Closed', 'Failed checks',
       'Awaiting acceptance', 'Grant Suspended', 'Payments Suspended',
       'Awaiting assessment', 'Successful', 'In Transfer', 'In Progress',
       'Created', 'Withdrawn', 'Offer returned by RO for amendments',
       'Suspended', 'Offer returned for amendments', 'Terminating',
       'Funded Off System', 'Returned', 'Deferred',
       'Finished Institute Project', 'Finished', 'Terminated',
       'Completed-Finished', 'Completed-Terminated',
       'Active Institute Project', 'DTA - Finished', 'Initiating',
       'Awarded', 'Live', 'Final Claim', 'On Hold', 'Offer Letter Sent',
       'Not set', 'Project Setup On Hold', 'Project Setup In Progress'],
      dtype=object)

In [8]:
# Write sql query
qry = "SELECT organisation_id, organisation_name, organisation_source FROM ukri_ssd_snapshot.2025_04_24_ssd_organisation;"

# Read sql query
organisation = pd.read_sql(qry, con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\714498904.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  organisation = pd.read_sql(qry, con)


In [9]:
organisation[['organisation_id', 'organisation_source']].duplicated().sum()

0

In [10]:
data_select.drop(columns='organisation_source_y', inplace=True)
data_select.rename(columns={'organisation_source_x':'organisation_source'}, inplace=True)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\898269222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select.drop(columns='organisation_source_y', inplace=True)
C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\898269222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select.rename(columns={'organisation_source_x':'organisation_source'}, inplace=True)


In [11]:
data_merge = pd.merge(data_select, organisation, on=['organisation_id', 'organisation_source'])


In [12]:
DSTL_pseudonyms = ["Defence Science & Tech Lab DSTL", "Defence Science and Technology Laboratory", "DEFENCE SCIENCE AND TECHNOLOGY LABORATORIES (DSTL)", "DSTL"]

National_physics_laboratory_pseudonyms = ["National Physical Laboratory NPL", "National Physical Laboratory", "NATIONAL PHYSICAL LABORATORY LIMITED", "NPL"]

data_select = data_merge.loc[data_merge['organisation_name'].isin(DSTL_pseudonyms) | data_merge['organisation_name'].isin(National_physics_laboratory_pseudonyms)]

In [13]:
# take NPL or DSTL opportunities

opps = data_select['opportunity_id'].drop_duplicates().to_list()

data_merge[data_merge['opportunity_id'].isin(opps)]

df_opps = df[df['opportunity_id'].isin(opps)]

In [14]:
df_opps # opportunities with NPL or DSTL as an organisation

# Select decision date is not null or in the future
data_opps_select = df_opps[~df_opps['decision_date'].isna() & (df_opps['decision_date'].dt.date < datetime.date.today()) & (df_opps['application_status'] != 'RO Withdrawn')]

In [16]:
data_opps_unique = data_opps_select[(data_opps_select['joint_proposal'] == 'No') | (data_opps_select['parent_award'] == 'Yes')]

In [17]:
data_opps_unique['app_ref_source'] = data_opps_unique['application_reference'] + '_' + data_opps_unique['application_source']

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\2816497502.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_opps_unique['app_ref_source'] = data_opps_unique['application_reference'] + '_' + data_opps_unique['application_source']


In [18]:
data_opps_unique['derived_status'].unique()

array(['Successful', 'Rejected'], dtype=object)

In [19]:
data_opps_unique['success_flag'] = 1

data_opps_unique.loc[data_opps_unique['derived_status'] == 'Rejected', 'success_flag'] = 0

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\1938546181.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_opps_unique['success_flag'] = 1


In [20]:
data_opps_group = data_opps_unique.groupby(['opportunity_id', 'opportunity_name', 'opportunity_source'], as_index=False).agg({'app_ref_source':'nunique', 'success_flag':['sum', 'mean']})


data_opps_group.columns = ['_'.join(col) for col in data_opps_group.columns]


In [21]:
data_opps_sort = data_opps_group.sort_values(by='success_flag_mean', ascending=False)



In [22]:
opps_non_competative = data_opps_sort[data_opps_sort['success_flag_mean'] > 0.75]['opportunity_id_'].to_list() # not competative

In [23]:
data_select # NPL and DSTL

# select only opportunities which were considered competative
data_select['competative'] = 'No'
data_select.loc[~data_select['opportunity_id'].isin(opps_non_competative), 'competative'] = 'Yes'

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\3429989841.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select['competative'] = 'No'


In [24]:
data_select['success_flag'] = 1

data_select.loc[data_select['derived_status'] == 'Rejected', 'success_flag'] = 0

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\4291790863.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select['success_flag'] = 1


In [25]:
data_select.loc[data_select['organisation_name'].isin(DSTL_pseudonyms), 'organisation_name'] = 'DSTL'
data_select.loc[data_select['organisation_name'].isin(National_physics_laboratory_pseudonyms), 'organisation_name'] = 'NPL'

In [26]:
data_select.groupby(['organisation_name', 'competative'], as_index=False).agg({'application_reference':'nunique', 'success_flag':'mean'}).to_clipboard(index=False)

In [27]:
data_select.groupby(['organisation_name', 'success_flag', 'competative'], as_index=False).agg({'amount_applied_for':'sum'}).to_clipboard(index=False)

In [28]:
data_select.groupby(['organisation_name', 'success_flag'], as_index=False).agg({'amount_applied_for':'sum'})

,organisation_name,success_flag,amount_applied_for
0,DSTL,0,4.619481e+06
1,NPL,0,1.653081e+07
2,NPL,1,1.420855e+07


In [29]:
data_select[data_select['organisation_name'] == 'DSTL']

,application_reference,application_source,decision_owner,decision_date,opportunity_id,opportunity_name,opportunity_source,derived_status,application_status,organisation_source,amount_applied_for,joint_proposal,parent_award,organisation_id,participant_id,organisation_role,organisation_name,competative,success_flag
23543,BB/P011675/1,Siebel,BBSRC,2016-11-16,1-3JSZW9,16CTP,Siebel,Rejected,Rejected,Siebel,NaN,No,No,1311,BB/P011675/1,Participant,DSTL,Yes,0
438109,16352,IUK - IFS,Innovate UK,2018-08-14,143,143 - Commercialising Quantum Devices: Innovat...,IUK,Rejected,Rejected,IUK,4519795.9,No,No,00158000012WRwBAAW,16352_16380,Participant,DSTL,Yes,0
543906,10119788,IUK - IFS,Innovate UK,2024-05-08,1844,"Knowledge Asset Grant Fund: Expand, Spring 2024",IUK,Rejected,Rejected,IUK,99685.0,No,No,001P3000008oihCIAQ,10119788_108529,Lead Participant,DSTL,Yes,0


In [30]:
data_select.groupby(['organisation_name'], as_index=False).agg({'amount_applied_for':'sum'})

,organisation_name,amount_applied_for
0,DSTL,4.619481e+06
1,NPL,3.073936e+07


In [31]:
data_select.groupby(['organisation_name', 'competative'], as_index=False).agg({'amount_applied_for':'sum'})

,organisation_name,competative,amount_applied_for
0,DSTL,Yes,4.619481e+06
1,NPL,No,5.756753e+06
2,NPL,Yes,2.498260e+07


In [32]:
spend = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_04_24_ssd_spend;", con)

C:\Users\MHil02\AppData\Local\Temp\ipykernel_13480\1137698173.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  spend = pd.read_sql("SELECT * FROM ukri_ssd_snapshot.2025_04_24_ssd_spend;", con)


In [33]:
data_select[['application_reference', 'application_source', 'success_flag', 'organisation_name']]

,application_reference,application_source,success_flag,organisation_name
5595,19350,TFS,0,NPL
9305,36053,TFS,1,NPL
11001,40817,TFS,1,NPL
12034,47867,TFS,0,NPL
12740,44005,TFS,1,NPL
...,...,...,...,...
520557,10128555,IUK - IFS,0,NPL
520558,10129076,IUK - IFS,0,NPL
520559,10131468,IUK - IFS,0,NPL
543906,10119788,IUK - IFS,0,DSTL


In [34]:
data_spend = pd.merge(spend, data_select[['application_reference', 'application_source', 'success_flag', 'organisation_name', 'decision_date']], on=['application_reference', 'application_source'], how='right', indicator=True)


In [35]:
data_spend[(data_spend['_merge'] == 'right_only') & (data_spend['success_flag'] == 1)]

,award_reference,transfer_reference,application_reference,paymentdate,paymentdate_financial_year,PaymentQuarterID,ExpenditureType,amount,type,application_source,funder,administrator,payment_type,success_flag,organisation_name,decision_date,_merge
1,NaN,NaN,36053,NaN,NaN,NaN,NaN,NaN,NaN,TFS,NaN,NaN,NaN,1,NPL,2024-10-01 13:11:45,right_only
2,NaN,NaN,40817,NaN,NaN,NaN,NaN,NaN,NaN,TFS,NaN,NaN,NaN,1,NPL,2024-12-09 16:58:30,right_only
684,NaN,NaN,25966,NaN,NaN,NaN,NaN,NaN,NaN,IUK - IFS,NaN,NaN,NaN,1,NPL,2018-11-07 00:00:00,right_only
736,NaN,NaN,10142097,NaN,NaN,NaN,NaN,NaN,NaN,IUK - IFS,NaN,NaN,NaN,1,NPL,2024-11-05 00:00:00,right_only
747,NaN,NaN,36666,NaN,NaN,NaN,NaN,NaN,NaN,IUK - IFS,NaN,NaN,NaN,1,NPL,2019-07-01 00:00:00,right_only
752,NaN,NaN,45435,NaN,NaN,NaN,NaN,NaN,NaN,IUK - IFS,NaN,NaN,NaN,1,NPL,2019-12-10 00:00:00,right_only
760,NaN,NaN,10032640,NaN,NaN,NaN,NaN,NaN,NaN,IUK - IFS,NaN,NaN,NaN,1,NPL,2022-04-27 00:00:00,right_only


In [36]:
data_spend = pd.merge(spend, data_select[['application_reference', 'application_source', 'success_flag', 'organisation_name', 'decision_date']], 
                      on=['application_reference', 'application_source'])


In [37]:
data_spend.groupby(['paymentdate_financial_year'], as_index=False).agg({'amount':'sum'}).to_clipboard(index=False)